In [1]:
import sys
sys.path.append(sys.path[0] + '/../')
sys.path.append(sys.path[0] + '/../models/davit/')

import torch
import torch.nn as nn
import time
from tqdm import tqdm
from torchsummary import summary
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from dataloaders.minc_dataloader import MINCDataset, MINCDataLoader
from utils.utils import CosineDecayLR, Metrics
from timm.models.davit import DaViT

['/home/ahmed/workspace/notebook/matrec/train', '', '/home/ahmed/workspace/ros2/install/rslidar_msg/lib/python3.8/site-packages', '/home/ahmed/workspace/ros2/install/kia_soul/lib/python3.8/site-packages', '/home/ahmed/workspace/ros2/install/dynamixel_sdk_custom_interfaces/lib/python3.8/site-packages', '/home/ahmed/workspace/ros2/install/can_msgs/lib/python3.8/site-packages', '/home/ahmed/workspace/ros2/install/campus_dae/lib/python3.8/site-packages', '/home/ahmed/workspace/ros2/install/apollo_msgs/lib/python3.8/site-packages', '/opt/ros/foxy/lib/python3.8/site-packages', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '/home/ahmed/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/local/lib/python3.8/dist-packages/yapf-0.32.0-py3.8.egg', '/usr/local/lib/python3.8/dist-packages/yacs-0.1.8-py3.8.egg', '/usr/local/lib/python3.8/dist-packages/tb_nightly-2.10.0a20220718-py3.8.egg', '/usr/local/lib/python3.8/dist-packages/iso

ModuleNotFoundError: No module named 'ssim'

In [ ]:
minc_path = '/home/ahmed/workspace/notebook/matrec/datasets/minc'
labels_path = '/home/ahmed/workspace/notebook/matrec/datasets/minc/train.txt'
labels_path_t = '/home/ahmed/workspace/notebook/matrec/datasets/minc/test.txt'
checkpoint = "../weights/davit_minc.pth"


BATCH_SIZE = 4
TRAIN_ITER = 10000
TEST_ITER = 400
LOAD = False
start = 0
lr = 0.00001
size = 256

train_dataloader = MINCDataLoader(minc_path, labels_path, batch_size=BATCH_SIZE, size=size, f=0.16)

dataset_test = MINCDataset(minc_path, labels_path_t, size=(size, size))
test_loader = DataLoader(dataset=dataset_test, batch_size=BATCH_SIZE, num_workers=0, pin_memory=False, shuffle=True)


model = DaViT(num_classes=23)

In [ ]:
if LOAD:
    model.load_state_dict(torch.load(checkpoint), strict=False)
model = model.train()
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
metrics = Metrics()

In [ ]:
#add augmentation
#rescaling [0, 1] and normalization (mean, std)
for epc in range(1,3):
    r = tqdm(range(start, len(train_dataloader)), leave=False, desc="Epoch {} starting from iteration {}: ".format(epc, start), 
                                                                     total=len(train_dataloader)-start)    
    
    for idx in r:
        x, y = train_dataloader[idx]
        y_pred = model(x.cuda())
        lf = metrics.compute(y_pred, y)
        lf.backward()
        optimizer.step()
        optimizer.zero_grad()
        r.set_postfix(loss=lf.item())
        
        if idx % TRAIN_ITER == 0 and idx != start:
            #save checkpoint
            torch.save(model.state_dict(), checkpoint)
            #test loss run on val
            with torch.no_grad():
                ac = 0
                for i in range(TEST_ITER):
                    x, y = next(iter(test_loader))
                    y_pred = model(x.cuda())
                    ac = ac + loss.accuracy(y_pred, y)
                ac = ac/TEST_ITER
                print('Test N ' + str(epc) + '-' + str(idx//TRAIN_ITER) + ' : ' + str(float(ac)) + '\n')
    start = 0
    torch.save(model.state_dict(), checkpoint)